<a href="https://www.kaggle.com/code/aadishchopra/disastertweets?scriptVersionId=124643683" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2/saved_model.pb
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2/keras_metadata.pb
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2/assets/vocab.txt
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2/variables/variables.index
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2/variables/variables.data-00000-of-00001


In [39]:
import torch
torch.cuda.is_available()

True

In [2]:
import datasets
import transformers
from transformers import AutoTokenizer,AutoModelForSequenceClassification,Trainer,TrainingArguments
from sklearn.model_selection import train_test_split

In [3]:
pd.set_option('display.max_colwidth', None)
df=pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')

In [4]:
df.sample()

,id,keyword,location,text,target
3571,5103,famine,NaN,Robert Conquest Famine Museum Kiev @GuidoFawkes @MediaGuido https://t.co/WE40iUX7Ib,0


# Remove non-ascii characters




In [8]:
#1. Remove characters other than ascii characters
df['new_text']=df['text'].apply(lambda x : x.encode('ascii',errors='ignore').decode())

In [5]:
pd.set_option('display.max_colwidth', None)
df_test=pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
df_test=pd.DataFrame(df_test,columns=['id','text'])
df_test['new_text']=df_test['text'].apply(lambda x : x.encode('ascii',errors='ignore').decode())
df_test.head()

,id,text,new_text
0,0,Just happened a terrible car crash,Just happened a terrible car crash
1,2,"Heard about #earthquake is different cities, stay safe everyone.","Heard about #earthquake is different cities, stay safe everyone."
2,3,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all","there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all"
3,9,Apocalypse lighting. #Spokane #wildfires,Apocalypse lighting. #Spokane #wildfires
4,11,Typhoon Soudelor kills 28 in China and Taiwan,Typhoon Soudelor kills 28 in China and Taiwan


In [6]:
df_test_dict=datasets.Dataset.from_dict(df_test)

In [9]:
df=df[['new_text','target']]
df.head()
df_train,df_eval=train_test_split(df,test_size=0.1)

In [10]:
df_dict=datasets.Dataset.from_dict(df_train)
df_eval_dict=datasets.Dataset.from_dict(df_eval)




In [11]:
tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased')

In [17]:
#tokenizer.encode_plus('try this string')
#>{'input_ids': [101, 3046, 2023, 5164, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

{'input_ids': [101, 3046, 2023, 5164, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [18]:
def letstokenize(text):
    text_t=text['new_text']
    target=text['target']
    encoding=tokenizer(text_t,padding="max_length",truncation=True,max_length=128)
    encoding["labels"]=target
    return encoding

encoded_df=df_dict.map(letstokenize)
encoded_eval_dict=df_eval_dict.map(letstokenize)

  0%|          | 0/6851 [00:00<?, ?ex/s]

  0%|          | 0/762 [00:00<?, ?ex/s]

In [28]:
disaster_bert=AutoModelForSequenceClassification.from_pretrained(
                                                                'bert-base-uncased',
                                                                 num_labels=2
                                                                )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [41]:
batch_size=8
metric_name='f1'
repo_name='kaggle-disaster-tweets'

args=TrainingArguments(
                  output_dir=repo_name,
                  evaluation_strategy='epoch',
                  metric_for_best_model=metric_name,
                  per_device_train_batch_size=batch_size,
                  num_train_epochs=5,
                  load_best_model_at_end=True,
                  #no_cuda=True,
                  per_device_eval_batch_size=batch_size,
                  weight_decay=0.05,
                  save_strategy='epoch',
                  learning_rate=5e-5,
                  report_to="none"
                  #push_to_hub=True
                    )

In [42]:
from datasets import load_metric
from sklearn.metrics import roc_auc_score,f1_score,accuracy_score

In [44]:
!pip install evaluate
import evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.2 MB/s eta 0:00:00


In [51]:
evaluate.enable_progress_bar()

In [58]:
def compute_metrics (eval_pred):
    metric = evaluate.load('f1')
    logits, labels = eval_pred
    preds = np.argmax(logits, axis = -1)
    return metric.compute(predictions=preds, references = labels)

In [59]:

trainer = Trainer(
    disaster_bert,
    args,
    train_dataset=encoded_df,
    eval_dataset=encoded_eval_dict,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
    )
     

In [60]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,F1
1,0.292600,0.801160,0.773984
2,0.274900,0.854625,0.765550
3,0.214400,0.889726,0.784848
4,0.151300,1.008833,0.785388
5,0.109600,1.105819,0.788606


TrainOutput(global_step=4285, training_loss=0.1995276024906332, metrics={'train_runtime': 597.4852, 'train_samples_per_second': 57.332, 'train_steps_per_second': 7.172, 'total_flos': 2253217300339200.0, 'train_loss': 0.1995276024906332, 'epoch': 5.0})

In [64]:
trainer.evaluate()

{'eval_loss': 1.1058194637298584,
 'eval_f1': 0.7886056971514244,
 'eval_runtime': 5.3558,
 'eval_samples_per_second': 142.276,
 'eval_steps_per_second': 17.925,
 'epoch': 5.0}

In [72]:
from transformers import TextClassificationPipeline
pipe = TextClassificationPipeline(model=disaster_bert, tokenizer=tokenizer, return_all_scores=False,device=0)
# outputs a list of dicts like [[{'label': 'NEGATIVE', 'score': 0.0001223755971295759},  {'label': 'POSITIVE', 'score': 0.9998776316642761}]]


/opt/conda/lib/python3.7/site-packages/transformers/pipelines/text_classification.py:107: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  UserWarning,


In [74]:
df_test['result']=pipe(df_test['new_text'].tolist())

In [77]:
df_test['target']=df_test['result'].apply(lambda x: int(x['label'][-1]))

In [78]:
df_test

,id,text,new_text,result,target
0,0,Just happened a terrible car crash,Just happened a terrible car crash,"{'label': 'LABEL_1', 'score': 0.9979619979858398}",1
1,2,"Heard about #earthquake is different cities, stay safe everyone.","Heard about #earthquake is different cities, stay safe everyone.","{'label': 'LABEL_1', 'score': 0.9981051683425903}",1
2,3,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all","there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all","{'label': 'LABEL_1', 'score': 0.9980998635292053}",1
3,9,Apocalypse lighting. #Spokane #wildfires,Apocalypse lighting. #Spokane #wildfires,"{'label': 'LABEL_1', 'score': 0.9981056451797485}",1
4,11,Typhoon Soudelor kills 28 in China and Taiwan,Typhoon Soudelor kills 28 in China and Taiwan,"{'label': 'LABEL_1', 'score': 0.9981095790863037}",1
...,...,...,...,...,...
3258,10861,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTENERS XrWn,EARTHQUAKE SAFETY LOS ANGELES SAFETY FASTENERS XrWn,"{'label': 'LABEL_1', 'score': 0.9981034994125366}",1
3259,10865,Storm in RI worse than last hurricane. My city&amp;3others hardest hit. My yard looks like it was bombed. Around 20000K still without power,Storm in RI worse than last hurricane. My city&amp;3others hardest hit. My yard looks like it was bombed. Around 20000K still without power,"{'label': 'LABEL_1', 'score': 0.998106837272644}",1
3260,10868,Green Line derailment in Chicago http://t.co/UtbXLcBIuY,Green Line derailment in Chicago http://t.co/UtbXLcBIuY,"{'label': 'LABEL_1', 'score': 0.998102605342865}",1
3261,10874,MEG issues Hazardous Weather Outlook (HWO) http://t.co/3X6RBQJHn3,MEG issues Hazardous Weather Outlook (HWO) http://t.co/3X6RBQJHn3,"{'label': 'LABEL_1', 'score': 0.9980695843696594}",1


In [80]:
submit=df_test[['id','target']]

In [81]:
submit.to_csv('submission.csv',index=False)